In [139]:
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import glob
from neuropy.utils.mixins.print_helpers import ProgressMessagePrinter
# from neuropy.io.openephysio import load_ttl_events
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.core import Position, Neurons, NeuronType
from neuropy.io import PhyIO
from neuropy.io import OptitrackIO

In [140]:
known_data_session_type_dict = {'kdiba':Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'),
                'bapun':Path('R:\data\Bapun\Day5TwoNovel'),
                'rachel':Path('R:\data\Rachel\merged_M1_20211123_raw_phy')}

active_path = known_data_session_type_dict['rachel']
# active_path = known_data_session_type_dict['bapun']

In [12]:
opti_folder = Path('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/behav_csv')
opti_data = OptitrackIO(opti_folder)

In [159]:
def _rach_prepare_for_Neurons_obj(neuron_cluster_info_df):
    ## Filter out all the zero elements
    # included_indicies = np.where(neuron_cluster_info_df['q']>0)
    included_indicies = neuron_cluster_info_df['q']>0
    neuron_cluster_info_df = neuron_cluster_info_df[included_indicies]
    # neuron_cluster_info_df = neuron_cluster_info_df[neuron_cluster_info_df['q']>0]
    neuron_cluster_info_df['neuron_type'] = NeuronType.from_qclu_series(neuron_cluster_info_df.q.astype('float'))
    return neuron_cluster_info_df, included_indicies

In [163]:
phydata = PhyIO(active_path)
np.shape(phydata.spiketrains) # (109,)
# phydata.shank_ids
phydata.cluster_info

active_neuron_cluster_info_df, included_indicies = _rach_prepare_for_Neurons_obj(phydata.cluster_info)
# active_neuron_cluster_info_df
# phydata.n_channels # 128
neuron_included_spiketrains = phydata.spiketrains[included_indicies]
# np.shape(neuron_included_spiketrains)

C:\Users\pho\AppData\Local\Temp\ipykernel_21348\1347062814.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neuron_cluster_info_df['neuron_type'] = NeuronType.from_qclu_series(neuron_cluster_info_df.q.astype('float'))


C:\Users\pho\AppData\Local\Temp\ipykernel_21348\1347062814.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neuron_cluster_info_df['neuron_type'] = NeuronType.from_qclu_series(neuron_cluster_info_df.q.astype('float'))


(107,)

# Normal Numpy Stuff

In [ ]:
def find_numpy_files(in_directory: Path):
    return list(glob.glob(str(active_path.joinpath('*.np[yz]'))))

def robust_load_npy_file(f):
    if isinstance(f, str):
        f = Path(f)
    if f.is_file():
        dict_rep = None
        try:
            dict_rep = np.load(f, allow_pickle=True)
        except NotImplementedError:
            print("Issue with pickled POSIX_PATH on windows for path {}, falling back to non-pickled version...".format(f))
            temp = pathlib.PosixPath
            pathlib.PosixPath = pathlib.PurePosixPath # Bad hack
            dict_rep = np.load(f, allow_pickle=True)
        try:
            dict_rep = dict_rep.item() # extract the single item if it's a scalar
        except ValueError:
            # print(f'dict_rep not an array of size 1: {dict_rep}')
            pass
        return dict_rep
    else:
        return None


active_numpy_files = find_numpy_files(active_path)
active_numpy_files

In [ ]:
numpy_vars = {}
for np_name in active_numpy_files:
    # print(f'{np_name}')
    np_path = Path(np_name)
    with ProgressMessagePrinter(np_name, 'Loading', f'{Path(np_name).stem}'):
        numpy_vars[np_name] = robust_load_npy_file(np_path)
        # numpy_vars[np_name] = np.load(np_name)

numpy_vars

In [ ]:
numpy_vars.keys()
{a_path:np.shape(loaded_item) for a_path, loaded_item in numpy_vars.items()}
pd.DataFrame({'file':list(numpy_vars.keys()), 'type':[type(loaded_item) for loaded_item in numpy_vars.values()], 'shape':[np.shape(loaded_item) for loaded_item in numpy_vars.values()]})
# pd.DataFrame({'file':list(numpy_vars.keys()), 'type':[type(loaded_item) for loaded_item in numpy_vars.values()], 'shape':[np.shape(loaded_item) for loaded_item in numpy_vars.values()]})

In [17]:
# Get dict objects
dict_only_numpy_vars = {a_path:loaded_item for a_path, loaded_item in numpy_vars.items() if isinstance(loaded_item, dict)}
# dict_only_numpy_vars

In [ ]:
a_dict_var = dict_only_numpy_vars['R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\merged_M1_20211123_raw.neurons.bak.npy']
{a_key:np.shape(loaded_item) for a_key, loaded_item in a_dict_var.items()}

In [ ]:
a_dict_var = dict_only_numpy_vars['R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\merged_M1_20211123_raw.neurons.npy']
{a_key:np.shape(loaded_item) for a_key, loaded_item in a_dict_var.items()}

In [27]:
a_dict_var = dict_only_numpy_vars['R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\merged_M1_20211123_raw.probegroup.npy']
print(f'a_dict_var: {a_dict_var}')
{a_key:np.shape(loaded_item) for a_key, loaded_item in a_dict_var.items()}

a_dict_var: {'data': Empty DataFrame
Columns: [x, y, contact_id, channel_id, shank_id, connected, probe_id]
Index: [], 'metadata': {}}


{'data': (0, 7), 'metadata': ()}

In [137]:
# 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\'
# r'R:\data\Rachel\merged_M1_20211123_raw_phy\cluster_info.tsv'
# ['cluster_info.tsv']
neuron_cluster_info_df = pd.read_csv(r'R:\data\Rachel\merged_M1_20211123_raw_phy\cluster_info.tsv', sep='\t')
# neuron_cluster_info_df

In [138]:
## Filter out all the zero elements
neuron_cluster_info_df = neuron_cluster_info_df[neuron_cluster_info_df['q']>0]
# neuron_cluster_info_df.reset_index(inplace=True)
neuron_cluster_info_df

,id,amp,ch,depth,fr,group,n_spikes,purity,q,sh
86,235,210.666351,17,2200.0,0.109639,good,421,1.000000,2,1.0
92,270,93.256348,88,780.0,1.218012,good,4677,0.678582,1,1.0
98,278,74.356003,18,2180.0,2.026374,good,7781,0.568974,8,1.0
103,291,78.024879,19,2160.0,5.771559,good,22162,0.546836,4,1.0
109,304,92.246445,19,2160.0,0.455225,good,1748,0.789999,2,1.0
...,...,...,...,...,...,...,...,...,...,...
590,1635,93.333519,48,1580.0,0.098181,good,377,0.699155,3,1.0
591,1636,147.557922,52,1500.0,4.986896,good,19149,0.917042,7,1.0
596,1666,145.966782,9,2360.0,0.840655,good,3228,0.898850,2,1.0
597,1667,79.362167,10,2340.0,0.411733,good,1581,0.898850,2,1.0


In [136]:
rachel_override_raw_neuronID_strings = pd.read_csv(r'R:\data\Rachel\merged_M1_20211123_raw_phy\merged_M1_20211123_raw_neuronID.csv', sep='\t')
rachel_override_raw_neuronType_strings = pd.read_csv(r'R:\data\Rachel\merged_M1_20211123_raw_phy\merged_M1_20211123_raw_neuronType.csv', sep='\t')
rachel_override_raw_neuronType_strings

,neuronType
0,other
1,pyr
2,pyr
3,mua
4,int
...,...
105,mua
106,other
107,pyr
108,pyr


In [ ]:
# neuronTypeLookupDict = {a_key:a_value for a_key, a_value in zip(rachel_override_raw_neuronID_strings.neuronID.to_numpy(), rachel_override_raw_neuronType_strings.neuronType.to_numpy())}
# neuronTypeLookupDict

In [100]:
# neuron_cluster_info_df['neuron_type'] = [neuronTypeLookupDict[int(q)] for q in neuron_cluster_info_df.q.to_numpy()]
# neuron_cluster_info_df

In [127]:
neuron_cluster_info_df['neuron_type'] = NeuronType.from_qclu_series(neuron_cluster_info_df.q.astype('float'))
# qclu_Series = neuron_cluster_info_df.q #.astype('float')
# temp_neuronTypeStrings = pd.cut(x=qclu_Series, bins=NeuronType.classCutoffValues(), labels=NeuronType.shortClassNames())
# temp_neuronTypeStrings
neuron_cluster_info_df

,level_0,index,id,amp,ch,depth,fr,group,n_spikes,purity,q,sh,neuron_type
0,0,86,235,210.666351,17,2200.0,0.109639,good,421,1.000000,2,1.0,NeuronType.PYRAMIDAL
1,1,92,270,93.256348,88,780.0,1.218012,good,4677,0.678582,1,1.0,NeuronType.PYRAMIDAL
2,2,98,278,74.356003,18,2180.0,2.026374,good,7781,0.568974,8,1.0,NeuronType.INTERNEURONS
3,3,103,291,78.024879,19,2160.0,5.771559,good,22162,0.546836,4,1.0,NeuronType.PYRAMIDAL
4,4,109,304,92.246445,19,2160.0,0.455225,good,1748,0.789999,2,1.0,NeuronType.PYRAMIDAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,102,590,1635,93.333519,48,1580.0,0.098181,good,377,0.699155,3,1.0,NeuronType.PYRAMIDAL
103,103,591,1636,147.557922,52,1500.0,4.986896,good,19149,0.917042,7,1.0,NeuronType.CONTAMINATED
104,104,596,1666,145.966782,9,2360.0,0.840655,good,3228,0.898850,2,1.0,NeuronType.PYRAMIDAL
105,105,597,1667,79.362167,10,2340.0,0.411733,good,1581,0.898850,2,1.0,NeuronType.PYRAMIDAL


In [ ]:
# neuron_cluster_info_df[neuron_cluster_info_df['neuron_type'] != 'other']

In [128]:
# ## Do filtering:
# active_neuron_cluster_info_df = neuron_cluster_info_df.copy()

# # replace 'int' with 'inter'
# active_neuron_cluster_info_df.loc[active_neuron_cluster_info_df['neuron_type'] == 'int', 'neuron_type'] = 'inter'
# active_neuron_cluster_info_df = active_neuron_cluster_info_df[active_neuron_cluster_info_df['neuron_type'] != 'other'] # filter out the 'other' types
# active_neuron_cluster_info_df

# Fix Rachel's Neurons.npy file:

In [ ]:
neurons_dict = dict_only_numpy_vars['R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\merged_M1_20211123_raw.neurons.bak.npy']
neurons_dict['neuron_ids'] = test_neurons_obj.neuron_ids[included_indicies]
neurons_dict['neuron_type'] = active_neuron_cluster_info_df.neuron_type.to_numpy()
neurons_dict['shank_ids'] = active_neuron_cluster_info_df.sh.to_numpy()
neurons_dict['spiketrains'] = neuron_included_spiketrains
neurons_dict
# neurons_dict['neuron_type'] = .astype(str)

In [ ]:
neurons_dict

In [175]:
test_neurons_obj = Neurons.from_dict(neurons_dict)
test_neurons_obj

Neurons
 n_neurons: 107
 n_total_spikes: 1058800
 t_start: 0.0
 t_stop: 7200

In [176]:
np.shape(test_neurons_obj.neuron_ids)

(107,)

In [ ]:
test_neurons_obj.neuron_type

In [177]:
test_neurons_obj.n_neurons # 109

107

In [178]:
np.shape(test_neurons_obj.spiketrains)

(107,)

In [179]:
np.shape(test_neurons_obj.neuron_ids) # (109,)

(107,)

In [180]:
np.shape(test_neurons_obj.neuron_type) # (107,)

(107,)

In [182]:
test_neurons_obj.filename = Path(r'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\merged_M1_20211123_raw.neurons.npy')
test_neurons_obj.save()

merged_M1_20211123_raw.neurons.npy saved
